# Scripts to mine, process and convert the LCD station data


> (Worked on this locally, hence the local paths.)










## After the LCD files were downloaded, we renamed the files, did some manipulations and some web page scraping to get additional data.

In [ ]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import csv

####################################################################################
# Renames the csvs to the station ids
####################################################################################

# Define the directory where your CSV files are located
csv_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Loop through each CSV file
for csv_file in csv_files:
    csv_path = os.path.join(csv_directory, csv_file)

    # Open the CSV file
    with open(csv_path, 'r') as file:
        # Read the first row (header) to determine the column names
        csv_reader = csv.reader(file)
        header = next(csv_reader)

        # Find the index of the "STATION" column
        station_column_index = header.index("STATION")

        # Read the last 5 characters from the "STATION" column value
        id_value = None
        for row in csv_reader:
            station_value = row[station_column_index]
            id_value = station_value[-5:]  # Extract the last 5 characters
            break  # We only need the first row

    if id_value:
        # Generate the new file name based on the extracted ID value
        new_file_name = f'{id_value}.csv'

        # Rename the CSV file
        new_csv_path = os.path.join(csv_directory, new_file_name)
        os.rename(csv_path, new_csv_path)

        print(f'Renamed {csv_file} to {new_file_name}')
    else:
        print(f'Unable to extract ID from {csv_file}')


In [ ]:
import pandas as pd

####################################################################################
# gets the difference between the 151 all station and my station count
####################################################################################


def compare_station_ids(csv_all_stations, csv_my_stations, output_csv):
    df_all = pd.read_csv(csv_all_stations, dtype={'STATION_ID': str})
    df_my = pd.read_csv(csv_my_stations, dtype={'STATION_ID': str})

    set_all = set(df_all['STATION_ID'])
    set_my = set(df_my['STATION_ID'])

    # Find station IDs that are different
    diff_all = set_all - set_my
    diff_my = set_my - set_all

    # Extract rows for differing station IDs from both DataFrames
    diff_all_df = df_all[df_all['STATION_ID'].isin(diff_all)]
    diff_my_df = df_my[df_my['STATION_ID'].isin(diff_my)]

    # Combine the two DataFrames
    diff_df = pd.concat([diff_all_df, diff_my_df], ignore_index=True)

    # Save the differences to a CSV file
    diff_df.to_csv(output_csv, index=False)
    print(f"Differences saved to {output_csv}.")

# Paths to your CSV files
csv_all_stations = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_ALL_STATIONS_ALL_INFO.csv'
csv_my_stations = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_MY_STATIONS_ALL_INFO.csv'
output_csv = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_STATION_DIFFERENCES.csv'

# Compare the station IDs and output the differences
compare_station_ids(csv_all_stations, csv_my_stations, output_csv)



In [ ]:
import pandas as pd

####################################################################################
# gets the list of station ids with leading zeros intact.
####################################################################################

# Replace the path with your actual file path
excel_file_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_ALL_STATIONS.xlsx'

# Read the Excel file
df = pd.read_excel(excel_file_path)

# Assuming 'STATION_ID' is the column name with the station IDs
station_ids = df['STATION_ID'].tolist()

# Convert the list into a string format with leading zeros intact, ready to be pasted into the scraping script
station_ids_string = ', '.join([f"'{str(id).zfill(5)}'" for id in station_ids])

# Print the formatted list of station IDs
print(f"station_ids = [{station_ids_string}]")



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

####################################################################################
#scrapes all 151 webpages to create a comprehensive file of all stations with their lat, long, name and elavation
####################################################################################

# List of station IDs as strings to maintain leading zeros
station_ids = [
    '23239', '94299', '24283', '23224', '23191', '23155', '23161', '93216',
    '03182', '00115', '23157', '23225', '23158', '93245', '00433', '23152',
    '23136', '03154', '03164', '03177', '23203', '93203', '93104', '03179',
    '00206', '23254', '53175', '24286', '23240', '53186', '03104', '53144',
    '23114', '23199', '03165', '93101', '24213', '53151', '23167', '93193',
    '93217', '03166', '53143', '00135', '00228', '53119', '03167', '93228',
    '00392', '93115', '03144', '93194', '03180', '03159', '23110', '00205',
    '23285', '23243', '23129', '00117', '53141', '93134', '23174', '53130',
    '93242', '03181', '23119', '93205', '00369', '93243', '23257', '23258',
    '23244', '03183', '24259', '23259', '24215', '93136', '93227', '23179',
    '00397', '93112', '23230', '53121', '03102', '93210', '93110', '93138',
    '23182', '23289', '93209', '00320', '00227', '93111', '23149', '53120',
    '24216', '04222', '24257', '03171', '00396', '23271', '23232', '23206',
    '23208', '93225', '23233', '23122', '93231', '93117', '03178', '23188',
    '93107', '03131', '23272', '23234', '93232', '23293', '93206', '93116',
    '93226', '23187', '93184', '53152', '23190', '23273', '93197', '23213',
    '93244', '04204', '00395', '93230', '23237', '53139', '00479', '03122',
    '03174', '23202', '00346', '93201', '93114', '93121', '23275', '00174',
    '93241', '23130', '93214', '23131', '93144', '23277', '53150'
]

# Initialize your dataframe with the updated column names
df = pd.DataFrame(columns=['STATION_ID', 'STATION_NAME', 'STATION_LATITUDE', 'STATION_LONGITUDE', 'STATION_ELEVATION'])

# Define the base URL
base_url = "https://www.ncei.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:"

# Function to scrape latitude, longitude, elevation, and station name
def scrape_station_data(station_id):
    # Construct the URL for the current station ID
    url = f"{base_url}{station_id}/detail"

    # Send a request to the URL
    response = requests.get(url)

    # If the request was successful, proceed to parse the data
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the 'Name' row and extract the station name
        name_td = soup.find('td', text='Name')
        station_name = name_td.find_next_sibling('td').text if name_td else ''

        # Find the 'Latitude/Longitude' row and extract the values
        lat_lon_td = soup.find('td', text='Latitude/Longitude')
        lat, lon = ('', '')
        if lat_lon_td:
            lat_lon_val_td = lat_lon_td.find_next_sibling('td')
            if lat_lon_val_td:
                lat_lon_text = lat_lon_val_td.text.strip()
                lat, lon = lat_lon_text.replace('°', '').split(',')

        # Find the 'Elevation' row and extract the value
        elevation_td = soup.find('td', text='Elevation')
        elevation = elevation_td.find_next_sibling('td').text.strip() if elevation_td else ''

        return station_name.strip(), lat.strip(), lon.strip(), elevation.strip()
    else:
        return '', '', '', ''

# Function to print messages in color
def print_success(message):
    print(f"\033[92m{message}\033[0m")  # Green text

def print_failure(message):
    print(f"\033[91m{message}\033[0m")  # Red text

# Iterate over each station ID and scrape the data
results = []
for station_id in station_ids:
    station_name, lat, lon, elevation = scrape_station_data(station_id)
    if station_name and lat and lon and elevation:
        results.append({
            'STATION_ID': station_id,
            'STATION_NAME': station_name,
            'STATION_LATITUDE': lat,
            'STATION_LONGITUDE': lon,
            'STATION_ELEVATION': elevation
        })
        print_success(f"{station_id}: SUCCESS")
    else:
        print_failure(f"{station_id}: FAIL")

# Convert the list of dictionaries to a DataFrame and concat with the main DataFrame
df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

# Define a valid path for your system
output_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_ALL_STATIONS_ALL_INFO.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save the DataFrame to a CSV file
print("Scraping complete. Saving to CSV...")
df.to_csv(output_path, index=False)
print(f"CSV file has been saved to {output_path}.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

####################################################################################
# gets all the files i downloaded and uses them to determine
# which webpages to scrape to create a comprehensive file of all MY stations with their lat, long, name and elavation
####################################################################################

# Function to get station IDs from file names in the given directory
def get_station_ids_from_files(directory_path):
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    station_ids = [f.split('.')[0] for f in csv_files]
    return station_ids

# Function to scrape latitude, longitude, elevation, and station name
def scrape_station_data(station_id):
    base_url = "https://www.ncei.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:"
    url = f"{base_url}{station_id}/detail"

    try:
        response = requests.get(url)
        response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code

    except requests.exceptions.HTTPError as errh:
        print("Http Error:", errh)
        return '', '', '', ''
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
        return '', '', '', ''
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
        return '', '', '', ''
    except requests.exceptions.RequestException as err:
        print("Oops: Something Else", err)
        return '', '', '', ''

    try:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract station name
        name_td = soup.find('td', text='Name')
        station_name = name_td.find_next_sibling('td').text.strip() if name_td else ''

        # Extract latitude and longitude
        lat_lon_td = soup.find('td', text='Latitude/Longitude')
        lat, lon = ('', '')
        if lat_lon_td:
            lat_lon_val_td = lat_lon_td.find_next_sibling('td')
            if lat_lon_val_td:
                lat_lon_text = lat_lon_val_td.text.strip()
                lat, lon = lat_lon_text.replace('°', '').split(',')
                # Clean up any extra whitespace or characters
                lat = lat.strip()
                lon = lon.strip().replace('W', '-').replace('E', '')  # Assuming Western Hemisphere for 'W'

        # Extract elevation
        elevation_td = soup.find('td', text='Elevation')
        elevation = elevation_td.find_next_sibling('td').text.strip() if elevation_td else ''

        return station_name, lat, lon, elevation

    except Exception as e:
        print(f"An error occurred while parsing data for station {station_id}: {e}")
        return '', '', '', ''

# Function to print messages in color
def print_success(message):
    print(f"\033[92m{message}\033[0m")  # Green text

def print_failure(message):
    print(f"\033[91m{message}\033[0m")  # Red text

# Path to the directory containing the CSV files
directory_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'

# Use the function to dynamically create a list of station IDs from CSV filenames in the directory
station_ids = get_station_ids_from_files(directory_path)

# Initialize dataframe with updated column names
df = pd.DataFrame(columns=['STATION_ID', 'STATION_NAME', 'STATION_LATITUDE', 'STATION_LONGITUDE', 'STATION_ELEVATION'])

# Iterate over each station ID and scrape the data
results = []
for station_id in station_ids:
    station_name, lat, lon, elevation = scrape_station_data(station_id)
    if station_name and lat and lon and elevation:
        results.append({
            'STATION_ID': station_id,
            'STATION_NAME': station_name,
            'STATION_LATITUDE': lat,
            'STATION_LONGITUDE': lon,
            'STATION_ELEVATION': elevation
        })
        print_success(f"{station_id}: SUCCESS")
    else:
        print_failure(f"{station_id}: FAIL")

# Convert the list of dictionaries to a DataFrame and concatenate with the main DataFrame
df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

# Define a valid path for your system
output_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/LCD_MY_STATIONS_ALL_INFO.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save the DataFrame to a CSV file
print("Scraping complete. Saving to CSV...")
df.to_csv(output_path, index=False)
print(f"CSV file has been saved to {output_path}.")

## We begin processing and combining the raw data.


> In this section I combined all the data into one file, I would then modify this approach to keep the station data files seperate. The section after this is where I take that alternative approach.



In [ ]:
import os
import pandas as pd
import glob

####################################################################################
# Check if all files have the same structure.
####################################################################################


# Directory containing individual CSV files
input_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'

# List of all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

# Initialize a dictionary to store the column information of each CSV
csv_structure = {}

# Go through each file and check the structure
for file in csv_files:
    try:
        # Read just the first row to get the column headers
        df = pd.read_csv(file, nrows=1)
        # Create a tuple of the column headers
        columns = tuple(df.columns.tolist())
        # Add the column structure to the dictionary with the filename as key
        csv_structure[file] = columns
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Check if all files have the same structure
first_file_columns = csv_structure[list(csv_structure.keys())[0]]
consistent_structure = all(columns == first_file_columns for columns in csv_structure.values())

# Print the results
if consistent_structure:
    print("All files have the same structure.")
else:
    print("Files have inconsistent structures. Please review the differences.")
    for file, columns in csv_structure.items():
        if columns != first_file_columns:
            print(f"File {file} has a different structure.")


All files have the same structure.


In [ ]:
# import os
# import pandas as pd
# import glob

# ####################################################################################
# # Combine into 1 file.
# ####################################################################################


# # Directory containing individual CSV files
# input_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'

# # Output directory for the combined CSV
# output_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA'
# output_filename = 'ALL_LCD_RAW_DATA_COMBINED.csv'
# output_filepath = os.path.join(output_directory, output_filename)

# # List of all CSV files in the input directory
# csv_files = glob.glob(os.path.join(input_directory, '*.csv'))
# print(f"Found {len(csv_files)} files to process.")

# # Initialize a list to hold dataframes
# dataframes = []

# # Loop through each file, read it into a pandas DataFrame, and add it to the list
# for i, file in enumerate(csv_files, 1):
#     try:
#         print(f"Processing file {i}/{len(csv_files)}: {file}")
#         df = pd.read_csv(file, low_memory=False)
#         dataframes.append(df)
#     except Exception as e:
#         print(f"Error processing {file}: {e}")

# # Combine all files in the list
# print("Combining files...")
# combined_csv = pd.concat(dataframes)

# # Export to CSV
# print(f"Writing combined data to {output_filepath}...")
# combined_csv.to_csv(output_filepath, index=False)

# print("All files have been combined and saved successfully.")




In [ ]:
# import os
# import pandas as pd
# import glob

# ####################################################################################
# # Check the length of the file and the individual files.
# ####################################################################################


# # Directory containing individual CSV files
# input_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'

# # Path to the combined CSV file
# combined_csv_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/ALL_LCD_RAW_DATA_COMBINED.csv'

# # List of all CSV files in the input directory
# csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

# # Calculate the sum of rows of all individual CSV files
# # Subtract 1 for the header row in each file
# total_rows_individual_files = sum(pd.read_csv(file, dtype=str, usecols=[0]).shape[0] for file in csv_files) - len(csv_files)

# # Calculate the number of rows in the combined CSV file
# # Here we assume the combined CSV also has a single header row
# combined_csv_rows = pd.read_csv(combined_csv_path, dtype=str, usecols=[0]).shape[0] - 1

# # Compare the numbers
# if total_rows_individual_files == combined_csv_rows:
#     print("The row counts match. The files have been combined successfully.")
# else:
#     print("The row counts do not match.")
#     print(f"Total rows in individual files (excluding headers): {total_rows_individual_files}")
#     print(f"Total rows in combined file (excluding header): {combined_csv_rows}")



In [ ]:
# import pandas as pd

# ####################################################################################
# # Remove extra columns from combined file and reformate date and the station id.
# ####################################################################################

# # Path to the combined CSV file
# combined_csv_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/ALL_LCD_RAW_DATA_COMBINED.csv'

# # New file path for the modified CSV file
# modified_csv_path = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/MODIFIED_LCD_RAW_DATA.csv'


# # Columns you want to keep
# columns_to_keep = [
#     'STATION', 'DATE', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
#     'HourlyPrecipitation', 'HourlyPresentWeatherType', 'HourlyRelativeHumidity',
#     'HourlySkyConditions', 'HourlyWetBulbTemperature', 'HourlyWindDirection',
#     'HourlyWindGustSpeed', 'HourlyWindSpeed'
# ]

# # Specify dtype as string for all columns to avoid DtypeWarning
# dtype_dict = {col: str for col in columns_to_keep}

# # Read the combined CSV file, selecting only the columns to keep, with specified data types
# df = pd.read_csv(combined_csv_path, usecols=columns_to_keep, dtype=dtype_dict)

# # Convert the 'DATE' column to datetime
# df['DATE'] = pd.to_datetime(df['DATE'])

# # Extract year, month, day, hour, and minute into separate columns
# df['YEAR'] = df['DATE'].dt.year
# df['MONTH'] = df['DATE'].dt.month
# df['DAY'] = df['DATE'].dt.day
# df['HOUR'] = df['DATE'].dt.hour  # Extract hour
# df['MINUTE'] = df['DATE'].dt.minute  # Extract minute

# # Extract the last 5 digits of the 'STATION' column to a new 'STATION_ID' column
# df['STATION_ID'] = df['STATION'].str[-5:]

# # Drop the original 'STATION' column if you no longer need it
# # df = df.drop('STATION', axis=1)

# # Reorder columns to have the date and time components next to each other, if desired
# df = df[['STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'HourlyDewPointTemperature',
#          'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyPresentWeatherType',
#          'HourlyRelativeHumidity', 'HourlySkyConditions', 'HourlyWetBulbTemperature',
#          'HourlyWindDirection', 'HourlyWindGustSpeed', 'HourlyWindSpeed']]

# # Write the modified DataFrame to a new CSV file
# df.to_csv(modified_csv_path, index=False)

# print(f"The modified file has been saved to {modified_csv_path}")


The modified file has been saved to /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/MODIFIED_LCD_RAW_DATA.csv


In [ ]:
# import pandas as pd
# from tqdm import tqdm

# ####################################################################################
# # Checking file length.
# ####################################################################################


# # File paths
# file_path_1 = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/ALL_LCD_RAW_DATA_COMBINED.csv'
# file_path_2 = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/MODIFIED_LCD_RAW_DATA.csv'

# # Define a chunk size
# chunk_size = 100000  # This means the CSV will be processed in chunks of 100,000 rows

# # Initialize a progress bar for each file without a total since we don't know the total number of rows
# chunks1 = pd.read_csv(file_path_1, chunksize=chunk_size, low_memory=False)
# chunks2 = pd.read_csv(file_path_2, chunksize=chunk_size, low_memory=False)

# # Process the first file with a progress bar
# num_rows_df1 = 0
# for chunk in tqdm(chunks1, desc='Reading first file'):
#     num_rows_df1 += chunk.shape[0]

# # Process the second file with a progress bar
# num_rows_df2 = 0
# for chunk in tqdm(chunks2, desc='Reading second file'):
#     num_rows_df2 += chunk.shape[0]

# # Print the number of rows
# print(f"Number of rows in the first file: {num_rows_df1}")
# print(f"Number of rows in the second file: {num_rows_df2}")


Reading first file: 217it [02:06,  1.71it/s]
Reading second file: 217it [00:11, 18.84it/s]

Number of rows in the first file: 21632463
Number of rows in the second file: 21632463


## Below we are working on modifying the files but without combining them.
> Originally we combined them into one file but we modified the approach to have each station's data contained in its own file and store them all in a folder.




In [ ]:
import pandas as pd
import os
import glob

####################################################################################
# Drop the extra columns from the 139 files and from the ones we keep get a tally of the data types
# they have. Move the modified files withot the extra columns to a new folder.
# Print the types of data in the columns.
####################################################################################

"""
Column 'STATION' data type examples:
  int: [74718703104.0, 72064500227.0, 74509023244.0, 72287493134.0, 72295303183.0]
  float: []
  str: ['A0705300346', 'A0704900320', 'A0685400115']

Column 'DATE' data type examples:
  int: []
  float: []
  str: ['2014-08-21T00:53:00', '2014-11-06T09:56:00', '2014-11-10T01:35:00', '2014-11-06T09:55:00', '2014-11-06T01:58:00']
"""

# Input and output directories
input_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download'
output_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod'

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# List of all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

# Columns to keep
columns_to_keep = [
    'STATION', 'DATE', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
    'HourlyPrecipitation', 'HourlyPresentWeatherType', 'HourlyRelativeHumidity',
    'HourlySkyConditions', 'HourlyWetBulbTemperature', 'HourlyWindDirection',
    'HourlyWindGustSpeed', 'HourlyWindSpeed'
]

# Initialize a dictionary to store column data types across files
column_data_types = {col: set() for col in columns_to_keep}

# Process each file
for file in csv_files:
    try:
        print(f"Processing file: {file}")

        # Read file with specified columns
        df = pd.read_csv(file, usecols=columns_to_keep, low_memory=False)

        # Check and record the data types of each column
        for col in df.columns:
            column_data_types[col].add(str(df[col].dtype))

        # Save to output directory
        output_file = os.path.join(output_directory, os.path.basename(file))
        df.to_csv(output_file, index=False)

        print(f"File saved: {output_file}")

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Display the data types found for each column across all files
for col, dtypes in column_data_types.items():
    print(f"Column '{col}' has data types: {dtypes}")


Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23131.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23131.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23119.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23119.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/03183.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/03183.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/03154.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/03154.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/93209.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DA

File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23225.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/93121.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/93121.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23190.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23190.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/00205.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/00205.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/53139.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/53139.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LC

File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23149.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23161.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23161.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/93107.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/93107.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23203.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23203.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/03104.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/03104.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LC

File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/03167.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/93206.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/93206.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/03166.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/03166.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/24283.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/24283.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download/23275.csv
File saved: /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod/23275.csv
Processing file: /Users/Elliot/Documents/Northeastern/6140/Project/LC

In [ ]:
import pandas as pd
import os
import glob


####################################################################################
# Folder containing the 139 relevant stations is further modified.
# Now they contain only the relevant features, and the date column is split down into more granular columns
# (year, month, day, hour, minute). Stripping the last 5 digits from the station column to
# get the station id as well.
####################################################################################


# Directory containing the CSV files to be modified
input_directory = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod'

# List of all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

# Process each file
for file in csv_files:
    try:
        # Read the file
        df = pd.read_csv(file, dtype=str)

        # Extract the last 5 characters from 'STATION' column for 'STATION_ID'
        df['STATION_ID'] = df['STATION'].str[-5:].str.zfill(5)

        # Convert 'DATE' column to datetime and extract components
        df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
        df['YEAR'] = df['DATE'].dt.year
        df['MONTH'] = df['DATE'].dt.month
        df['DAY'] = df['DATE'].dt.day
        df['HOUR'] = df['DATE'].dt.hour
        df['MINUTE'] = df['DATE'].dt.minute

        # Reorder and select columns as specified
        df = df[['STATION_ID', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'HourlyDewPointTemperature',
                 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyPresentWeatherType',
                 'HourlyRelativeHumidity', 'HourlySkyConditions', 'HourlyWetBulbTemperature',
                 'HourlyWindDirection', 'HourlyWindGustSpeed', 'HourlyWindSpeed']]

        # Save the modified DataFrame back to the same file
        df.to_csv(file, index=False)

    except Exception as e:
        print(f"Error processing {file}: {e}")

print("Processing complete.")


Processing complete.


## Need to go back and convert LCD into daily measurements.


> After we had the LCD files we realized we need to go back and convert the LCD files to daily measurements to match GHCN and the other datasets.



In [ ]:
import pandas as pd
import os
import numpy as np

####################################################################################
# Process 139 files in lcd mod to
# /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed
# we make the mixed data types a single data type aside from
# sky conditions and weather type which are complex strings and nan floats
####################################################################################


def preprocess_lcd_data(file_path):
    # Load the CSV with low_memory=False to prevent DtypeWarnings
    lcd_data = pd.read_csv(file_path, low_memory=False)

    # List of columns with mixed types (string representations of numbers and special characters)
    mixed_type_cols = ['HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
                       'HourlyPrecipitation', 'HourlyWindGustSpeed', 'HourlyWindSpeed',
                       'HourlyRelativeHumidity', 'HourlyWetBulbTemperature', 'HourlyWindDirection']

    # Convert string representations to numeric and handle special characters
    for col in mixed_type_cols:
        # Remove trailing 's', convert to numeric, and replace non-numeric with NaN
        lcd_data[col] = pd.to_numeric(lcd_data[col].astype(str).str.rstrip('s').replace('', np.nan), errors='coerce')

    # Handle special characters in non-numeric string columns
    lcd_data['HourlyPresentWeatherType'] = lcd_data['HourlyPresentWeatherType'].replace(['*', 'VRB'], np.nan)
    lcd_data['HourlySkyConditions'] = lcd_data['HourlySkyConditions'].replace('*', np.nan)

    # Additional preprocessing steps if required

    return lcd_data

lcd_data_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod'
preprocessed_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed'
os.makedirs(preprocessed_folder, exist_ok=True)

for file_name in os.listdir(lcd_data_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(lcd_data_folder, file_name)
        preprocessed_lcd_data = preprocess_lcd_data(file_path)
        output_file_path = os.path.join(preprocessed_folder, file_name)
        preprocessed_lcd_data.to_csv(output_file_path, index=False)
        print(f"Preprocessed and saved data for {file_name}")


Preprocessed and saved data for 23131.csv
Preprocessed and saved data for 23119.csv
Preprocessed and saved data for 03183.csv
Preprocessed and saved data for 03154.csv
Preprocessed and saved data for 93209.csv
Preprocessed and saved data for 23285.csv
Preprocessed and saved data for 03182.csv
Preprocessed and saved data for 23130.csv
Preprocessed and saved data for 93197.csv
Preprocessed and saved data for 24259.csv
Preprocessed and saved data for 03180.csv
Preprocessed and saved data for 23244.csv
Preprocessed and saved data for 23293.csv
Preprocessed and saved data for 03181.csv
Preprocessed and saved data for 00117.csv
Preprocessed and saved data for 93227.csv
Preprocessed and saved data for 93232.csv
Preprocessed and saved data for 23254.csv
Preprocessed and saved data for 93144.csv
Preprocessed and saved data for 23136.csv
Preprocessed and saved data for 93193.csv
Preprocessed and saved data for 23122.csv
Preprocessed and saved data for 03179.csv
Preprocessed and saved data for 93

In [ ]:
import pandas as pd
import os
from collections import defaultdict

####################################################################################
# Compare the data types of each column from the pre and post processed 139 files.
####################################################################################

# Directory containing the LCD data files
lcd_data_dir_processed = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed'
lcd_data_dir_unprocessed = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod'

# Dictionary to hold data type information for both folders
data_types_info_processed = defaultdict(set)
data_types_info_unprocessed = defaultdict(set)

# Function to process a directory and update data type information
def process_directory(directory, data_types_info):
    # List all CSV files in the directory
    lcd_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    total_files = len(lcd_files)

    # Process each file in the directory
    for idx, filename in enumerate(lcd_files):
        file_path = os.path.join(directory, filename)
        try:
            # Read the file into a DataFrame
            lcd_data = pd.read_csv(file_path, low_memory=False)

            # Update data type information for each column
            for col in lcd_data.columns:
                data_types_info[col].update(set(lcd_data[col].apply(type).unique()))

        except Exception as e:
            print(f"Error processing {filename}: {e}")

        # Print progress every 50 files
        if (idx + 1) % 50 == 0 or idx + 1 == total_files:
            print(f"Processed {idx + 1}/{total_files} files in {directory}")

# Process both directories
process_directory(lcd_data_dir_processed, data_types_info_processed)
process_directory(lcd_data_dir_unprocessed, data_types_info_unprocessed)

# Display the aggregated data type information for both folders
print("Processed Folder (Preprocessed):")
for col, types in data_types_info_processed.items():
    print(f"Column: {col}, Data Types: {types}")

print("\nUnprocessed Folder:")
for col, types in data_types_info_unprocessed.items():
    print(f"Column: {col}, Data Types: {types}")


Processed 50/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed
Processed 100/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed
Processed 139/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed
Processed 50/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod
Processed 100/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod
Processed 139/139 files in /Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod
Processed Folder (Preprocessed):
Column: STATION_ID, Data Types: {<class 'int'>}
Column: YEAR, Data Types: {<class 'int'>}
Column: MONTH, Data Types: {<class 'int'>}
Column: DAY, Data Types: {<class 'int'>}
Column: HOUR, Data Types: {<class 'int'>}
Column: MINUTE, Data Types: {<class 'int'>}
Column: HourlyDewPoint

In [ ]:
shared_features = {
    'Precipitation': {
        'lcd_col': 'HourlyPrecipitation',
        'ghcn_col': 'PRCP',
        'processing': 'sum'  # Sum of hourly precipitation for daily total
    },
    'Temperature': {
        'lcd_col': 'HourlyDryBulbTemperature',
        'ghcn_cols': ['TMAX', 'TMIN', 'TAVG'],
        'processing': ['max', 'min', 'mean']  # Max, Min, and Average of hourly temperatures
    },
    'DewPointTemperature': {
        'lcd_col': 'HourlyDewPointTemperature',
        'ghcn_col': 'ADPT',
        'processing': 'mean'  # Average of hourly dew point temperatures
    },
    'WetBulbTemperature': {
        'lcd_col': 'HourlyWetBulbTemperature',
        'ghcn_col': 'AWBT',
        'processing': 'mean'  # Average of hourly wet bulb temperatures
    },
    'WindSpeed': {
        'lcd_col': 'HourlyWindSpeed',
        'ghcn_col': 'AWND',
        'processing': 'mean'  # Average of hourly wind speeds
    },
    'RelativeHumidity': {
        'lcd_col': 'HourlyRelativeHumidity',
        'ghcn_cols': ['RHAV', 'RHMN', 'RHMX'],
        'processing': ['mean', 'min', 'max']  # Average, Minimum, and Maximum of hourly relative humidity
    }
}


In [ ]:
# Each feature's daily value is directly used as it is already in a daily format.
unique_ghcn_features = ['SNOW', 'SNWD', 'EVAP', 'FMTM', 'FRGB', 'FRGT', 'FRTH']


unique_lcd_features = {
    'MaxWindSpeed': {
        'lcd_col': 'HourlyWindSpeed',
        'processing': 'max'  # Maximum of hourly wind speeds
    },
    'WindDirectionMode': {
        'lcd_col': 'HourlyWindDirection',
        'processing': 'mode'  # Most common wind direction
    },
    'WindDirectionAverage': {
        'lcd_col': 'HourlyWindDirection',
        'processing': 'mean'  # Average of hourly wind directions (with circular data handling)
    },
    'MaxWindGust': {
        'lcd_col': 'HourlyWindGustSpeed',
        'processing': 'max'  # Maximum of hourly wind gust speeds
    },
    'WeatherTypeMode': {
        'lcd_col': 'HourlyPresentWeatherType',
        'processing': 'mode'  # Most common weather type
    },
    'SkyConditionMode': {
        'lcd_col': 'HourlySkyConditions',
        'processing': 'mode'  # Most common sky condition
    }
}


In [ ]:
import pandas as pd
import os
import numpy as np

####################################################################################
# Convert the 139 post processed files into a new converted folder that converts
# the hourly measurments to daily min, max, averages and modes.
####################################################################################

def calculate_circular_mean(angles):
    # Remove NaN values from angles
    angles = angles.dropna()

    # If no valid angles, return NaN
    if len(angles) == 0:
        return np.nan

    # Convert angles from degrees to radians
    angles_rad = np.radians(angles)

    # Calculate sine and cosine components
    sin_components = np.sin(angles_rad)
    cos_components = np.cos(angles_rad)

    # Calculate mean vector components
    avg_sin = np.mean(sin_components)
    avg_cos = np.mean(cos_components)

    # Calculate the average wind direction from the mean vector components
    avg_wind_direction = np.degrees(np.arctan2(avg_sin, avg_cos))

    # Adjust the range from -180° to 180° to 0° to 360°
    if avg_wind_direction < 0:
        avg_wind_direction += 360

    return avg_wind_direction

def mode_function(series):
    mode_result = series.mode()
    return mode_result.iloc[0] if not mode_result.empty else np.nan

def custom_dew_point_mean(series):
    DEW_POINT_TEMP_RANGE = (-22, 86)  # in °F
    filtered_series = series[(series >= DEW_POINT_TEMP_RANGE[0]) & (series <= DEW_POINT_TEMP_RANGE[1])]
    return filtered_series.mean()

def custom_wind_speed_mean(series):
    WIND_SPEED_RANGE = (0, 200)  # in mph, using your provided range
    filtered_series = series[(series >= WIND_SPEED_RANGE[0]) & (series <= WIND_SPEED_RANGE[1])]
    return filtered_series.mean()

def custom_max(series):
    WIND_SPEED_RANGE = (0, 200)  # in mph
    filtered_series = series[(series >= WIND_SPEED_RANGE[0]) & (series <= WIND_SPEED_RANGE[1])]
    return filtered_series.max()


def process_lcd_file(file_path):
    lcd_data = pd.read_csv(file_path)
    lcd_data['HOUR'] = pd.to_datetime(lcd_data['HOUR'], format='%H').dt.hour

    agg_functions = {
        'HourlyPrecipitation': 'sum',
        'HourlyDryBulbTemperature': ['max', 'min', 'mean'],
        'HourlyDewPointTemperature': custom_dew_point_mean,
        'HourlyWetBulbTemperature': 'mean',
        'HourlyWindSpeed': [custom_max, custom_wind_speed_mean],
        'HourlyRelativeHumidity': ['max', 'min', 'mean'],
        'HourlyWindDirection': [calculate_circular_mean, mode_function],
        'HourlyWindGustSpeed': 'max',
        'HourlyPresentWeatherType': mode_function,
        'HourlySkyConditions': mode_function
    }

    daily_data = lcd_data.groupby(['YEAR', 'MONTH', 'DAY']).agg(agg_functions).reset_index()


    # Flatten MultiIndex columns and rename
    new_columns = []
    for col in daily_data.columns:
        if isinstance(col, tuple):
            # Prefix the aggregation function name and append the rest of the column name (without 'Hourly')
            new_column_name = col[1] + col[0].replace('Hourly', '')
            new_columns.append(new_column_name)
        else:
            new_columns.append(col)
    daily_data.columns = new_columns

    daily_data = daily_data.rename(columns={
        'custom_dew_point_meanDewPointTemperature': 'meanDewPointTemperature',
        'custom_maxWindSpeed': 'maxWindSpeed',
        'custom_wind_speed_meanWindSpeed': 'meanWindSpeed'
    })

    return daily_data

lcd_data_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed'
processed_converted_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed_converted'
os.makedirs(processed_converted_folder, exist_ok=True)

for file_name in os.listdir(lcd_data_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(lcd_data_folder, file_name)
        daily_lcd_data = process_lcd_file(file_path)
        output_file_path = os.path.join(processed_converted_folder, file_name)
        daily_lcd_data.to_csv(output_file_path, index=False)
        print(f"Processed and saved daily data for {file_name}")


Processed and saved daily data for 23131.csv
Processed and saved daily data for 23119.csv
Processed and saved daily data for 03183.csv
Processed and saved daily data for 03154.csv
Processed and saved daily data for 93209.csv
Processed and saved daily data for 23285.csv
Processed and saved daily data for 03182.csv
Processed and saved daily data for 23130.csv
Processed and saved daily data for 93197.csv
Processed and saved daily data for 24259.csv
Processed and saved daily data for 03180.csv
Processed and saved daily data for 23244.csv
Processed and saved daily data for 23293.csv
Processed and saved daily data for 03181.csv


/var/folders/9b/w58m9pbs7dsdp2l04rxrznlw0000gn/T/ipykernel_17902/246626639.py:59: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lcd_data = pd.read_csv(file_path)


Processed and saved daily data for 00117.csv
Processed and saved daily data for 93227.csv
Processed and saved daily data for 93232.csv
Processed and saved daily data for 23254.csv
Processed and saved daily data for 93144.csv
Processed and saved daily data for 23136.csv
Processed and saved daily data for 93193.csv
Processed and saved daily data for 23122.csv
Processed and saved daily data for 03179.csv
Processed and saved daily data for 93230.csv
Processed and saved daily data for 93231.csv
Processed and saved daily data for 03144.csv
Processed and saved daily data for 93225.csv
Processed and saved daily data for 03178.csv
Processed and saved daily data for 23257.csv
Processed and saved daily data for 23243.csv
Processed and saved daily data for 00115.csv
Processed and saved daily data for 93184.csv
Processed and saved daily data for 53175.csv
Processed and saved daily data for 23152.csv
Processed and saved daily data for 93134.csv
Processed and saved daily data for 23191.csv


/var/folders/9b/w58m9pbs7dsdp2l04rxrznlw0000gn/T/ipykernel_17902/246626639.py:59: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lcd_data = pd.read_csv(file_path)


Processed and saved daily data for 23224.csv
Processed and saved daily data for 23230.csv
Processed and saved daily data for 93242.csv
Processed and saved daily data for 93243.csv
Processed and saved daily data for 23225.csv
Processed and saved daily data for 93121.csv
Processed and saved daily data for 23190.csv
Processed and saved daily data for 00205.csv
Processed and saved daily data for 53139.csv
Processed and saved daily data for 23179.csv
Processed and saved daily data for 23233.csv
Processed and saved daily data for 93241.csv
Processed and saved daily data for 24213.csv
Processed and saved daily data for 23232.csv
Processed and saved daily data for 23187.csv


...

Processed and saved daily data for 00227.csv
Processed and saved daily data for 53119.csv
Processed and saved daily data for 53121.csv
Processed and saved daily data for 23149.csv
Processed and saved daily data for 23161.csv
Processed and saved daily data for 93107.csv
Processed and saved daily data for 23203.csv
Processed and saved daily data for 03104.csv
Processed and saved daily data for 23202.csv
Processed and saved daily data for 00395.csv
Processed and saved daily data for 93112.csv
Processed and saved daily data for 23174.csv
Processed and saved daily data for 53120.csv
Processed and saved daily data for 93138.csv
Processed and saved daily data for 93110.csv
Processed and saved daily data for 93104.csv


/var/folders/9b/w58m9pbs7dsdp2l04rxrznlw0000gn/T/ipykernel_17902/246626639.py:59: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lcd_data = pd.read_csv(file_path)


Processed and saved daily data for 00397.csv
Processed and saved daily data for 00369.csv
Processed and saved daily data for 00433.csv


/var/folders/9b/w58m9pbs7dsdp2l04rxrznlw0000gn/T/ipykernel_17902/246626639.py:59: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lcd_data = pd.read_csv(file_path)


Processed and saved daily data for 00396.csv
Processed and saved daily data for 93111.csv
Processed and saved daily data for 23188.csv
Processed and saved daily data for 53150.csv
Processed and saved daily data for 53144.csv
Processed and saved daily data for 23110.csv
Processed and saved daily data for 04222.csv
Processed and saved daily data for 23272.csv
Processed and saved daily data for 93214.csv
Processed and saved daily data for 93228.csv
Processed and saved daily data for 93201.csv
Processed and saved daily data for 03174.csv
Processed and saved daily data for 23273.csv
Processed and saved daily data for 53151.csv
Processed and saved daily data for 53186.csv
Processed and saved daily data for 23271.csv
Processed and saved daily data for 23259.csv
Processed and saved daily data for 93203.csv
Processed and saved daily data for 24286.csv
Processed and saved daily data for 03177.csv
Processed and saved daily data for 93216.csv
Processed and saved daily data for 23258.csv
Processed 

In [ ]:
import pandas as pd
import os
import numpy as np

####################################################################################
# This is just a script that compares the converted daily interval files to the pre converted
# hourly/minute interval files
####################################################################################

def calculate_circular_mean(angles):
    angles = angles.dropna()
    if len(angles) == 0:
        return np.nan
    angles_rad = np.radians(angles)
    avg_sin = np.mean(np.sin(angles_rad))
    avg_cos = np.mean(np.cos(angles_rad))
    avg_wind_direction = np.degrees(np.arctan2(avg_sin, avg_cos))
    return avg_wind_direction if avg_wind_direction >= 0 else avg_wind_direction + 360

def mode_function(series):
    mode_result = series.mode()
    return mode_result.iloc[0] if not mode_result.empty else np.nan

def compare_hourly_daily(file_name, hourly_folder, daily_folder, year, month, day):
    # Load hourly data
    hourly_data = pd.read_csv(os.path.join(hourly_folder, file_name))
    # Load daily data
    daily_data = pd.read_csv(os.path.join(daily_folder, file_name))

    # Filter data for the specified year, month, and day
    hourly_data_specific_day = hourly_data[(hourly_data['YEAR'] == year) & (hourly_data['MONTH'] == month) & (hourly_data['DAY'] == day)]
    daily_data_specific_day = daily_data[(daily_data['YEAR'] == year) & (daily_data['MONTH'] == month) & (daily_data['DAY'] == day)]

    # Define aggregation functions
    agg_functions = {
        'HourlyPrecipitation': 'sum',
        'HourlyDryBulbTemperature': ['max', 'min', 'mean'],
        'HourlyDewPointTemperature': 'mean',
        'HourlyWetBulbTemperature': 'mean',
        'HourlyWindSpeed': ['max', 'mean'],
        'HourlyRelativeHumidity': ['max', 'min', 'mean'],
        'HourlyWindDirection': calculate_circular_mean,
        'HourlyWindGustSpeed': 'max',
        'HourlyPresentWeatherType': mode_function,
        'HourlySkyConditions': mode_function
    }

    # Perform aggregation
    daily_aggregated_specific_day = hourly_data_specific_day.groupby(['YEAR', 'MONTH', 'DAY']).agg(agg_functions).reset_index()

    # Rename columns in daily_aggregated_specific_day to match daily_data_specific_day
    daily_aggregated_specific_day.columns = [
        'YEAR', 'MONTH', 'DAY', 'sumPrecipitation', 'maxDryBulbTemperature',
        'minDryBulbTemperature', 'meanDryBulbTemperature', 'meanDewPointTemperature',
        'meanWetBulbTemperature', 'maxWindSpeed', 'meanWindSpeed', 'maxRelativeHumidity',
        'minRelativeHumidity', 'meanRelativeHumidity', 'calculate_circular_meanWindDirection',
        'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions'
    ]

    # Print column names for debugging
    print("Manually Aggregated Columns:")
    print(daily_aggregated_specific_day.columns.to_list())
    print("\nDaily CSV Columns:")
    print(daily_data_specific_day.columns.to_list())

    # Compare the values for the specific day
    print(f"\nComparing for file: {file_name} on {year}-{month}-{day}")
    for col in daily_aggregated_specific_day.columns:
        if col in daily_data_specific_day.columns:
            print(f"{col}: {daily_aggregated_specific_day[col].values[0]} | {daily_data_specific_day[col].values[0]}")

hourly_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed'
daily_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed_converted'
compare_hourly_daily('00115.csv', hourly_folder, daily_folder, year=2023, month=11, day=3)


Manually Aggregated Columns:
['YEAR', 'MONTH', 'DAY', 'sumPrecipitation', 'maxDryBulbTemperature', 'minDryBulbTemperature', 'meanDryBulbTemperature', 'meanDewPointTemperature', 'meanWetBulbTemperature', 'maxWindSpeed', 'meanWindSpeed', 'maxRelativeHumidity', 'minRelativeHumidity', 'meanRelativeHumidity', 'calculate_circular_meanWindDirection', 'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions']

Daily CSV Columns:
['YEAR', 'MONTH', 'DAY', 'sumPrecipitation', 'maxDryBulbTemperature', 'minDryBulbTemperature', 'meanDryBulbTemperature', 'meanDewPointTemperature', 'meanWetBulbTemperature', 'maxWindSpeed', 'meanWindSpeed', 'maxRelativeHumidity', 'minRelativeHumidity', 'meanRelativeHumidity', 'calculate_circular_meanWindDirection', 'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions']

Comparing for file: 00115.csv on 2023-11-3
YEAR: 2023 | 2023
MONTH: 11 | 11
DAY: 3 | 3
sumPrecipitation: 0.0 | 0.0
maxDryBulbTemperature: 68.0 | 6

In [ ]:
import pandas as pd
import os
import numpy as np


####################################################################################
# This is just a script that compares the converted daily interval files to the pre converted
# hourly/minute interval files
####################################################################################

def calculate_circular_mean(angles):
    angles = angles.dropna()
    if len(angles) == 0:
        return np.nan
    angles_rad = np.radians(angles)
    avg_sin = np.mean(np.sin(angles_rad))
    avg_cos = np.mean(np.cos(angles_rad))
    avg_wind_direction = np.degrees(np.arctan2(avg_sin, avg_cos))
    return avg_wind_direction if avg_wind_direction >= 0 else avg_wind_direction + 360

def mode_function(series):
    mode_result = series.mode()
    return mode_result.iloc[0] if not mode_result.empty else np.nan

def compare_hourly_daily(file_name, hourly_folder, daily_folder, year, month, day):
    # Load hourly data
    hourly_data = pd.read_csv(os.path.join(hourly_folder, file_name))
    # Load daily data
    daily_data = pd.read_csv(os.path.join(daily_folder, file_name))

    # Filter data for the specified year, month, and day
    hourly_data_specific_day = hourly_data[(hourly_data['YEAR'] == year) & (hourly_data['MONTH'] == month) & (hourly_data['DAY'] == day)]
    daily_data_specific_day = daily_data[(daily_data['YEAR'] == year) & (daily_data['MONTH'] == month) & (daily_data['DAY'] == day)]

    # Define aggregation functions
    agg_functions = {
        'HourlyPrecipitation': 'sum',
        'HourlyDryBulbTemperature': ['max', 'min', 'mean'],
        'HourlyDewPointTemperature': 'mean',
        'HourlyWetBulbTemperature': 'mean',
        'HourlyWindSpeed': ['max', 'mean'],
        'HourlyRelativeHumidity': ['max', 'min', 'mean'],
        'HourlyWindDirection': [calculate_circular_mean, mode_function],  # Added mode calculation
        'HourlyWindGustSpeed': 'max',
        'HourlyPresentWeatherType': mode_function,
        'HourlySkyConditions': mode_function
    }

    # Perform aggregation
    daily_aggregated_specific_day = hourly_data_specific_day.groupby(['YEAR', 'MONTH', 'DAY']).agg(agg_functions).reset_index()

    # Flatten MultiIndex columns and rename to match daily_data_specific_day
    new_columns = []
    for col in daily_aggregated_specific_day.columns:
        if isinstance(col, tuple):
            new_column_name = col[1] + col[0].replace('Hourly', '')
            new_columns.append(new_column_name)
        else:
            new_columns.append(col)
    daily_aggregated_specific_day.columns = new_columns

    # Print column names for debugging
    print("Manually Aggregated Columns:")
    print(daily_aggregated_specific_day.columns.to_list())
    print("\nDaily CSV Columns:")
    print(daily_data_specific_day.columns.to_list())

    # Compare the values for the specific day
    print(f"\nComparing for file: {file_name} on {year}-{month}-{day}")
    for col in daily_aggregated_specific_day.columns:
        if col in daily_data_specific_day.columns:
            print(f"{col}: {daily_aggregated_specific_day[col].values[0]} | {daily_data_specific_day[col].values[0]}")

hourly_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed'
daily_folder = '/Users/Elliot/Documents/Northeastern/6140/Project/LCD_DATA/Raw_data_download_mod_preprocessed_converted'
compare_hourly_daily('00115.csv', hourly_folder, daily_folder, year=2023, month=11, day=3)


Manually Aggregated Columns:
['YEAR', 'MONTH', 'DAY', 'sumPrecipitation', 'maxDryBulbTemperature', 'minDryBulbTemperature', 'meanDryBulbTemperature', 'meanDewPointTemperature', 'meanWetBulbTemperature', 'maxWindSpeed', 'meanWindSpeed', 'maxRelativeHumidity', 'minRelativeHumidity', 'meanRelativeHumidity', 'calculate_circular_meanWindDirection', 'mode_functionWindDirection', 'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions']

Daily CSV Columns:
['YEAR', 'MONTH', 'DAY', 'sumPrecipitation', 'maxDryBulbTemperature', 'minDryBulbTemperature', 'meanDryBulbTemperature', 'meanDewPointTemperature', 'meanWetBulbTemperature', 'maxWindSpeed', 'meanWindSpeed', 'maxRelativeHumidity', 'minRelativeHumidity', 'meanRelativeHumidity', 'calculate_circular_meanWindDirection', 'mode_functionWindDirection', 'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions']

Comparing for file: 00115.csv on 2023-11-3
YEAR: 2023 | 2023
MONTH: 11 | 11
DAY: 3 | 3